In [22]:
import os
import csv
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Flatten,Dense,Lambda, Convolution2D, Cropping2D, Dropout

print('Loading data...')
lines = []
with open('K:/AI/SDCND/term1/CarSimGitHub/CarND-Behavioral-Cloning-P3-master/windows_sim/data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

images =[]
measurements = []

#PreProcessing
#Upon inspecting the recorded data, we found that an extremely high number of recorded steering values were 0.0 as we ideally
#want to drive in the middle of road
#Hence we look out for images corresponding to that 0.0 steering angle, and choose to select all values except for the 0 values
#So when the model drives, it doesn't converge at 0 as the most optimal output due to overpopulation of 0

Offset = 1.2

for x in lines:
    if (x[3]!="0"):
        image_path=x[0]
        image=cv2.imread(image_path,cv2.IMREAD_COLOR)
        images.append(image)
        measurements.append((float(x[3])*Offset))       
    else:
        pass

X_train = np.array(images)
Y_train = np.array(measurements)

augmented_images, augmented_measurements = [], []
for image, measurement in zip(images,measurements):
    augmented_images.append(image)
    augmented_measurements.append(measurement)
    augmented_images.append(cv2.flip(image,1))
    augmented_measurements.append(measurement*(-1.0))

X_train = np.array(augmented_images)
Y_train = np.array(augmented_measurements)

print('Data Loaded. The data size after flipping is')
print(len(Y_train))

Loading data...
Data Loaded. The data size after flipping is
390


In [23]:
def resize_img(input):
    from keras.backend import tf as ktf
    return ktf.image.resize_images(input, (60, 140))

In [26]:
from keras.models import Sequential
from keras.layers import Flatten,Dense,Lambda,Convolution2D, Cropping2D, Dropout
model=Sequential()



#Preprocessing:
#Changing mean and standard deviation so that the model scales properly. We multiply it by a contrasting factor to 
# increase the contrast and thus increase the deviation
                                   
model.add(Lambda(lambda x: ((x / 255.0) - 0.5), input_shape=(160,320,3)))
#Cropping the unnecessary parts like : sky, trees, hood of car, ground off the road
model.add(Cropping2D(cropping=((60,21),(70,70))))

# re-size inside the model
model.add(Lambda(resize_img))

#NVidia Model
#Find more about this at https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/
model.add(Convolution2D(24,5,5,subsample=(1,2),activation='relu'))
model.add(Convolution2D(36,5,5,subsample=(2,2),activation='relu'))
model.add(Convolution2D(48,5,5,activation='relu'))
model.add(Convolution2D(64,3,3,subsample=(2,2),activation='relu'))
model.add(Convolution2D(64,3,3,activation='relu'))
model.add(Flatten())

model.add(Dense(1164))
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

model.compile(loss='mse',optimizer='adam')
model.fit(X_train,Y_train,validation_split=0.01,batch_size = 64,shuffle=True,nb_epoch=5)
model.summary()
model.save('model.h5')

#first i Travelled twice clockwise and twice anticlockwise
#However that data was insufficient to handle the sharp curves and bridge
#I noted where the car steered towards on these curves and bridge, went in to training mode and recorded me correcting my
#path from these wrong positions the car previously steered towards. This allowed the model to learn about what to do
#when it messes up its steering at odd paths, the result with this was : car offshooted at the right curve

#Next I plan on experimenting more with the acitvations functions, right now i have a relu at all the convolutions
#and one relu at the first fully connected layer. This time I introduce elu instead. Results : disastrous, the car drove off 
#the road immediately

#I switch back the relus and instead add a tanh for the first fully connected layer, car smashes at diverting point after bridge

#No activations in the FCCs, result : the car blocks on bridge and right turn

#Recapturing data


#increase the size of validation set to 3%
#and Try removing dropout, result : overshoots on first left turn

#dropout added for all layers, result is bad. The output is frozen at -1.29

#dropout only to be for convolution layers, car again overshoots. Get rid of dropouts over all.

#all dropouts removed, first left turn specifically trained. Results : much better, stops at the first left.

#Removing all dropouts, much better results but the car blocks at the first left

#Removing all relus from convolutions as well, not good at all. The car immediately goes off the road

#Retraining the first left turn, and relus added back. Result : blocks at bridge

#retraining the entire loop with left>right>left>right instead of using mouse. Trained only one loop in counter clockwise,
#results : first left wasn't clean but the car crossed it and it stuck on the next left turn

#retrained it for the second left turn. Results : in general the car sticks to the left

#retraining the entire set again. One turn clockwise and one anti clockwise. Results : not good


#retrained with one loop. Removed steering noise, removed contrast enhancement, removed subsample of 2x2 from first Conv layers
#result : the computer hanged, maybe because of the 2x2 subsample. Let's add it back to alternate layers. Result :
#overshoots at first left

#restore the previous data loading and segmenting method; three if loops. See if the result is similar, Result : Great results
#the first track was completed with a little errors in turns. The car fared surprisingly well on the tougher track too

#trained a few more cases of sharp curves, and also added the 2x contrasting multiplier, epochs = 10. Result : blocked at bridge

#Remove the 2x multiplier, epochs = 5. Result : not good.

#So we record all at once, once again . Result : Things were pretty smooth until the first left.

#We retrain that particular bit. Result is worse, I think we should record all the data in one go. now

#recording everything in one go, pressed keys for as short duration as possible and tapped them quickly,
#result : HUUUGE dataset of 18k, result : Not any good, had issues with going straight and also while taking right turns.

#Now I try to train it on the tougher track, try to allign it to the middle lane line as that is a contrasting area which the network
#can learn to follow. Result on the tougher track :  COMMENDABLE RESULTS!!! the whole track was completed, though there were 
#some overshoots at the sharpest turns of the track(which can be corrected by specefically training them
#, but overall it was pretty smooth. Result on the easier track : Overshoots on the second left. I've observed that it has
#a tendency to drive over the lane, probably because it learned to do that from the center lane marking from the tougher track.

#I'll train the sharp curves and also train the model to recover from it's excessive right orientation on the road and some dropout
#result : the output was frozen, definitely because of dropout. Have to retrain without it now

#without dropout, result : it doesn't perform that good on the easy track, but it's quite good on the tougher one. I retrained
#the slight overshoots further

#right now I just want to create a model for the tough track since it doesn't seem so tough given my recent progress
#result after retraining the overshoots : First overshoot is absolutely corrected! The car overshoots at a cliff

#retraining a new model for easier track. For training i do wiggly zig zag driving for only half the track, result : 
#performed quite good for its dataset size haha. 

#let's try the wiggly motion approach for the entire track. Result : did quite good, overshot at second left but that's correctable

#correcting the second left overshot, results : it's going a little too much on the left in general case

#added a 2x contrasting multiplier, results : the wiggling was enhanced

#thus i should train it using stable driving and then add a multiplier, results : it drove out on the left
#training a round in opposite direction result : quite good, just a little allignement issues 

#retrained those allighnement problem regions, result : overshot after the first left. 

#try with removing the 2x multiplier, added a 1.5 multiplier to incoming data and 1.1 multiplier to recorded steering measurments
#result : very good! though there's a slight offset to the right which can be trained.

#after offset training : results : works like a charm, minor block at pre bridge and at bridge. Let's reduce contrast multiplier
#and see how it goes without training anything extra

#try removing the contrast multiplier : results :  went out, seems like contrast multiplier enhances the sensitivity, and that felt like it was missing here
#Val_loss = 0.087

#we observe that the validation loss is always about double the training loss, Hence we have an overfitting model. 
#Therefore we increase epochs(second), increase validation set(first), add regularization(first)
#Val_loss =   0.1391     Result = There's a strong similarity in the wavy driving pattern. Overshoots on the first left.

#we play with contrast and offset multipliers. Remove the offset because that mismatches the training data.
#we add contrast of 1.2 and add regularization to all the layers
#Val_loss = 0.5      Result = Training is quite good, val_loss is only slightly above training loss. Track result : output froze
#to a single steering angle, just like it did with dropout

#turns out we can't really stop overfitting, we might use softer values but otherwise things might just freeze up

#remove all regularization. Used a 1.2x contrast.2x Steering Result : Things are shaky but it has survived all of it. Can probably
#retrain with smoother data and then use a 2x multiplier. Saving this model as Model_s2_c1.2

#Turns out that we don't really need to worry about losses for this project, as shows on multiple occasions, trying to reduce
#the overfit only leads to a frozen fixed steering angle.

#Retraining with smoother data. 1x contrast 1.5x steering. Great results. There's a lack of bridge data as we just went straight
#on it, adding some data for the bridge

#results : contrast multiplier decreases results. We might even change it to something less than one to improve results.

#added back image flipping to further enhance the model, Steering multiplier is 1.5x as the dataset is big now so the model knows
#most of its way around the track. Result : is a little wavy here and there but it gets the job done! 
#there's clearly more room for improvement.

#if things are same or better, try increasing the epochs


Train on 386 samples, validate on 4 samples
Epoch 1/5
386/386 [==============================] - 20s - loss: 0.1367 - val_loss: 0.0246
Epoch 2/5
386/386 [==============================] - 15s - loss: 0.0618 - val_loss: 0.0212
Epoch 3/5
386/386 [==============================] - 15s - loss: 0.0369 - val_loss: 0.0165
Epoch 4/5
386/386 [==============================] - 15s - loss: 0.0320 - val_loss: 0.0125
Epoch 5/5
386/386 [==============================] - 15s - loss: 0.0262 - val_loss: 0.0131
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_10 (Lambda)               (None, 160, 320, 3)   0           lambda_input_6[0][0]             
____________________________________________________________________________________________________
cropping2d_6 (Cropping2D)        (None, 79, 180, 3)    0           lambda_10[0][0]               